In [76]:
import tweepy
import re
from tweepy import OAuthHandler
from textblob import TextBlob
import pandas as pd

In [77]:
class TwitterClient(object):
    
    def __init__(self):
        
        consumer_key = '95tYh7LPK14kOIC1IUGOUb5vY'
        consumer_secret = 'ORp6reaiAPBrlIahaqdHLo7KxMRy6s5IgQBRcvnjr4Eyt99bFY'
        access_token = '1080412991793889281-XlZ9X9zSKU4NYsOOSQIb2QL3uFpbuJ'
        access_token_secret = 'T2T5Nq3PoEDJNOHssIBUSD6gQyi85zcnPaIZMMjYrhkcF'
        
        try:
            self.auth = OAuthHandler(consumer_key,consumer_secret)
            self.auth.set_access_token(access_token,access_token_secret)
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return (' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()))
    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
  
    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 
            #keep only those tweets which are in english
            f_tweets = [tweet for tweet in fetched_tweets if tweet.lang == "en"]
  
            # parsing tweets one by one 
            for tweet in f_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 
  

In [78]:
def main(): 
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    # calling function to get tweets 
    tweets = api.get_tweets(query = 'Donald Trump', count = 200) 
  
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    # percentage of neutral tweets 
    print("Neutral tweets percentage: {} % ".format(100*len(tweets) - len(ntweets) - len(ptweets)/len(tweets))) 
  
    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:10]: 
        print(tweet['text']) 
  
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:10]: 
        print(tweet['text']) 
  


In [79]:
#created_an_empty_dataframe
df = pd.DataFrame()

In [80]:
#tweets returned as per query and number of tweets
client = TwitterClient()
tweets = client.get_tweets(query = 'india',count = 100)

In [81]:
len(tweets)

71

In [82]:
df

""


In [83]:
#add tweet and its sentiment to dataframe
for i in tweets:
    df = df.append([[i['text'],i['sentiment']]],ignore_index  = True)

In [84]:
#name teh columns of dataframe
df.columns = ['tweet','sentiment']

In [85]:
def encode(text):
    return text.encode('utf-8')

In [86]:
#so that dataframe can be written to csv file without unicode error
df['tweet_2'] = df['tweet'].apply(encode)

In [87]:
df_2 = df.drop(['tweet'],axis = 1)

In [88]:
#final dataframe
df_2.head(3)

,sentiment,tweet_2
0,positive,RT @shampadhar: Fabulous idea putting India’s ...
1,positive,RT @rsprasad: Total value of BHIM-UPI transact...
2,neutral,RT @darshanpathak: A tale of two interviews (i...


In [90]:
#writing dataframe to a csv file
df_2.to_csv("G:/tweets.csv" , index = False)